In [1]:
import pandas as pd
import glob
from datetime import datetime, timedelta

In [12]:
file_list = glob.glob('sacct_*.lst')
df_list = []
sum_rows = 0
print("hello")
for file in file_list:
    df = pd.read_csv(file, delimiter='|', low_memory=False,on_bad_lines='skip')
    sum_rows += df.shape[0]
    print(df.shape[0])
    df_list.append(df)
# file_path = 'sacct_2024-09-01_2024-09-02_X.lst'

print(sum_rows)
# data = pd.read_csv(file_path, delimiter='|')

25824
17589
22092
19709
22532
17389
12931
20429
21664
40965
43672
38638
26560
13271
15330
10507
9175
18052
87320
38496
13927
40878
33620
43157
30638
30559
12246
17198
24555
768923


In [13]:
combined_df = pd.concat(df_list, ignore_index=True, join='outer')
summary_stats = combined_df.describe()

In [26]:
summary_stats = combined_df.describe()
combined_df['CPUTimeRAW'] = pd.to_numeric(combined_df['CPUTimeRAW'], errors='coerce')
combined_df['AllocCPUS'] = pd.to_numeric(combined_df['AllocCPUS'], errors='coerce')
combined_df['AllocNodes'] = pd.to_numeric(combined_df['AllocNodes'], errors='coerce')

# Group by 'User' and aggregate CPUTime and WallTime (example columns)
grouped_summary = combined_df.groupby('User').agg({
    'CPUTimeRAW': 'sum',
    'AllocCPUS':'sum',
    'AllocNodes':'sum',
    'JobID': 'count'
})
grouped_summary = grouped_summary.rename(columns={'JobID': 'Total Number of Jobs'})

In [27]:
def convert_seconds(seconds):
    days, remainder = divmod(seconds, 86400)   # 86400 seconds in a day
    hours, remainder = divmod(remainder, 3600) # 3600 seconds in an hour
    minutes, seconds = divmod(remainder, 60)   # 60 seconds in a minute
    return f"{int(days)}:{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

# Applying this to the 'CPUTimeRAW' column after summing
grouped_summary['CPUTime'] = grouped_summary['CPUTimeRAW'].apply(convert_seconds)


In [28]:
print(grouped_summary)

           CPUTimeRAW  AllocCPUS  AllocNodes  JobID        CPUTime
User                                                              
a_coe          8890.0        4.0         2.0      2     0:02:28:10
aarobins      73455.0       11.0         6.0      6     0:20:24:15
aaron593  126845184.0    14060.0       263.0    283  1468:02:46:24
aaronkho  106546016.0    68585.0      1079.0   1088  1233:04:06:56
aarons42     993972.0      106.0        26.0     27    11:12:06:12
...               ...        ...         ...    ...            ...
zikai      20602802.0      980.0       929.0    929   238:11:00:02
zimi          98964.0       20.0        19.0     20     1:03:29:24
zinelis      131285.0      699.0        17.0     57     1:12:28:05
ziqili      3086314.0      301.0        97.0    141    35:17:18:34
zqshen      1631874.0     1286.0       148.0    150    18:21:17:54

[508 rows x 5 columns]
